In [2]:
from cng.utils import *

duckdb_install_h3()

import ibis
from ibis import _
con = ibis.duckdb.connect(extensions = ["spatial", "h3"])

import streamlit as st
set_secrets(con, st.secrets["MINIO_KEY"], st.secrets["MINIO_SECRET"])

parquet = "https://minio.carlboettiger.info/public-biodiversity/pad-us-4/pad-us-4.parquet"
con.raw_sql(f"CREATE  OR REPLACE VIEW pad4 AS SELECT Unit_Nm, geom FROM '{parquet}'")

#parquet = "https://minio.carlboettiger.info/public-data/social-vulnerability/2022/SVI2022_US_tract.parquet" # faster with local
#con.raw_sql(f"CREATE  OR REPLACE VIEW svi AS SELECT FIPS, Shape AS geom FROM '{parquet}'")

In [3]:
# many multi-polygons
con.sql(f'select Unit_Nm, ST_NumGeometries(geom) AS n from pad4 LIMIT 5').execute()


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Unit_Nm,n
0,Laguna Cartagena National Wildlife Refuge,1
1,Servidumbre de Conservación Finca María Luisa,2
2,Servidumbre Escénica Montes Oscuros,10
3,Servidumbre de Conservación Siembra Tres Vidas,1
4,Servidumbre de Conservación Centro Espríritu S...,1


In [4]:
# test case
con.sql(f'''
WITH t1 AS (
SELECT Unit_Nm, 
       ST_Dump(geom) AS geom 
FROM pad4 LIMIT 5)
SELECT Unit_Nm, UNNEST(geom).geom FROM t1
''').execute()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Unit_Nm,(unnest(geom)).geom
0,Laguna Cartagena National Wildlife Refuge,"POLYGON ((-67.10254 18.00731, -67.10250 18.006..."
1,Servidumbre de Conservación Finca María Luisa,"POLYGON ((-67.10363 17.99423, -67.10309 17.994..."
2,Servidumbre de Conservación Finca María Luisa,"POLYGON ((-67.08988 17.98585, -67.08985 17.985..."
3,Servidumbre Escénica Montes Oscuros,"POLYGON ((-66.27890 18.06995, -66.27151 18.069..."
4,Servidumbre Escénica Montes Oscuros,"POLYGON ((-66.24115 18.04572, -66.24158 18.044..."
5,Servidumbre Escénica Montes Oscuros,"POLYGON ((-66.30271 18.01735, -66.30289 18.017..."
6,Servidumbre Escénica Montes Oscuros,"POLYGON ((-66.30415 18.01854, -66.30427 18.018..."
7,Servidumbre Escénica Montes Oscuros,"POLYGON ((-66.28831 18.01846, -66.28827 18.018..."
8,Servidumbre Escénica Montes Oscuros,"POLYGON ((-66.29271 18.01361, -66.29286 18.013..."
9,Servidumbre Escénica Montes Oscuros,"POLYGON ((-66.29476 18.01378, -66.29474 18.013..."


In [114]:
zoom = 11

con.sql(f'''
WITH t1 AS (
  SELECT Unit_Nm, ST_Dump(geom) AS geom 
  FROM pad4
) 
SELECT Unit_Nm, 
       h3_polygon_wkt_to_cells_string(UNNEST(geom).geom, {zoom}) AS cells
FROM t1
''').to_parquet(f"s3://public-biodiversity/pad-us-4/pad-h3-z{zoom}.parquet")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Unit_Nm,cells
0,Laguna Cartagena National Wildlife Refuge,"[8b4cc0b51a11fff, 8b4cc0b4242cfff, 8b4cc0b420e..."
1,Servidumbre de Conservación Finca María Luisa,"[8b4cc0b5c54bfff, 8b4cc0b5c40bfff, 8b4cc0b5c0d..."
2,Servidumbre de Conservación Finca María Luisa,"[8b4cc0b58aaafff, 8b4cc0b59c81fff, 8b4cc0b58da..."
3,Servidumbre Escénica Montes Oscuros,"[8b4cead904dcfff, 8b4cead9249cfff, 8b4cead9641..."
4,Servidumbre Escénica Montes Oscuros,"[8b4cead91534fff, 8b4cead90264fff, 8b4cead91c8..."
5,Servidumbre Escénica Montes Oscuros,"[8b4cc536339afff, 8b4cc53632b4fff, 8b4cc536339..."
6,Servidumbre Escénica Montes Oscuros,[]
7,Servidumbre Escénica Montes Oscuros,"[8b4cc536ececfff, 8b4cc536e191fff, 8b4cc536ed6..."
8,Servidumbre Escénica Montes Oscuros,[8b4cc536e575fff]
9,Servidumbre Escénica Montes Oscuros,[]
